In [ ]:
import warnings
warnings.filterwarnings("ignore")
import streamlit as st
import pandas as pd
import json
import re
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
from snowflake.snowpark import types as T
from snowflake.snowpark import Row
from snowflake.core import Root
from snowflake.cortex import Complete
session = get_active_session()


In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream('@aicollege.public.transcripts/Phase2.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=400)

In [ ]:
-- View the list of files in the stage to ensure 6 are present
LIST @AICOLLEGE.PUBLIC.TRANSCRIPTS PATTERN='.*\.pdf$';

In [ ]:
-- View a transcript
SELECT
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    '@AICOLLEGE.PUBLIC.TRANSCRIPTS',
    'Acme Corp Services Discovery Meeting Transcript.pdf',
    OBJECT_CONSTRUCT('mode','layout')
  ) AS layout;

### 🧾 Extract PDF Transcript Content with `PARSE_DOCUMENT`

We’ll use the [`PARSE_DOCUMENT`](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex) function to extract the full contents of each transcript PDF.

Snowflake has simplified working with unstructured documents by providing a Cortex AI SQL function that returns the extracted content as a structured JSON object — no external parsing tools required.

This is the **first step** in working with unstructured data, enabling:
- RAG pipelines powered by Cortex Search
- LLM workflows like summarization, translation, or classification
- Structured output extraction from forms and contracts

💡 `PARSE_DOCUMENT` supports two modes:
- **OCR mode** → Best for clean text extraction from scanned documents  
- **LAYOUT mode** → Best for preserving structure like tables, headers, and sections (used here)

We’ll store the extracted content along with customer name and file path in a `PARSED_TRANSCRIPTS` table for future use.

In [ ]:
-- Create the parsed_transcripts table
CREATE OR REPLACE TABLE PARSED_TRANSCRIPTS (
    CUSTOMER_NAME  STRING,                      -- e.g. “Acme Corp Services”
    RELATIVE_PATH  STRING PRIMARY KEY,          -- full path in the stage
    RAW_TEXT       VARIANT,                     -- PDF text
    LOADED_AT      TIMESTAMP_TZ DEFAULT CURRENT_TIMESTAMP()
);

-- Process files one at a time to avoid stage scanning issues
INSERT INTO PARSED_TRANSCRIPTS (CUSTOMER_NAME, RELATIVE_PATH, RAW_TEXT)
SELECT 
    REGEXP_REPLACE(RELATIVE_PATH,' Discovery Meeting Transcript\\.pdf$','') AS CUSTOMER_NAME,
    RELATIVE_PATH,
    SNOWFLAKE.XXX(      --> Use Snowflake's Parse Document function
        '@AICOLLEGE.PUBLIC.TRANSCRIPTS',
        RELATIVE_PATH,
        OBJECT_CONSTRUCT('mode','XXX') --> Use layout mode
    ) AS RAW_TEXT
FROM DIRECTORY('@AICOLLEGE.PUBLIC.TRANSCRIPTS') f
WHERE
  RELATIVE_PATH LIKE '%Discovery Meeting Transcript.pdf'
  AND RELATIVE_PATH NOT IN (SELECT RELATIVE_PATH FROM PARSED_TRANSCRIPTS);

In [ ]:
-- Validate PARSED_TRANSCRIPTS was created correctly
SELECT * FROM PARSED_TRANSCRIPTS LIMIT 5;

### ✂️ Chunk Transcript Text with SPLIT_TEXT_RECURSIVE_CHARACTER
Once we’ve extracted the full layout of each transcript, the next step is to split the long text into smaller overlapping chunks. This is a critical step before embedding or indexing for semantic search.

We’ll use the [`SPLIT_TEXT_RECURSIVE_CHARACTER`](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex) function, which is designed to split long documents intelligently based on a preferred delimiter (e.g., paragraph breaks).

This prepares the data for downstream use with:
- Cortex Search indexing
- Embedding generation
- Summarization or classification with LLMs

💡 We’re using:
- A chunk size of **800 characters**
- An overlap of **150 characters**
- A preferred break on "\n\n" to preserve paragraph structure

In [ ]:
-- Create chunks of the parsed transcripts
CREATE OR REPLACE TABLE TRANSCRIPT_CHUNKS (
    CUSTOMER_NAME  STRING,
    RELATIVE_PATH  STRING,
    CHUNK          STRING
);

-- Insert chunked content using SPLIT_TEXT_RECURSIVE_CHARACTER
INSERT INTO TRANSCRIPT_CHUNKS (CUSTOMER_NAME, RELATIVE_PATH, CHUNK)
WITH text_chunks AS (
    SELECT
        CUSTOMER_NAME,
        RELATIVE_PATH,
        SNOWFLAKE.XXX(  --> Use Snowflake's Split Text Recursive function
            RAW_TEXT:content::STRING,  -- extract string content
            'markdown',                -- tokenizer
            800,                       -- chunk size
            100,                       -- overlap
            ARRAY_CONSTRUCT('\n\n')    -- preferred break
        ) AS CHUNKS
    FROM PARSED_TRANSCRIPTS
    WHERE RAW_TEXT:content IS NOT NULL -- optional safety check
)
SELECT
    CUSTOMER_NAME,
    RELATIVE_PATH,
    chunk.value::STRING AS XXX  --> Save chunks of parsed transcripts as "CHUNK"
FROM text_chunks,
LATERAL FLATTEN(input => CHUNKS) AS chunk;  --> Flatten the chuncked output

In [ ]:
-- Validate TRANSCRIPT_CHUNKS was created correctly
SELECT * FROM TRANSCRIPT_CHUNKS LIMIT 5;

### 🔍 Make Your Transcripts Searchable with Cortex Search

Now that we've extracted and chunked the content of each transcript, the next step is to make it **searchable** using Snowflake's fully managed retrieval system: **Cortex Search**.

[`CORTEX SEARCH SERVICE`](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview) enables **hybrid retrieval** — combining vector search (semantic meaning) with keyword search (lexical matching) — to deliver **highly accurate results** out of the box.

This capability is a key enabler for Retrieval-Augmented Generation (RAG), allowing you to:
- Ask contextual questions over your documents using SQL or LLM interfaces
- Power chatbots and copilots with document-grounded responses
- Serve precise and explainable enterprise search across large volumes of unstructured content

💡 Powered by `snowflake-arctic-embed-l-v2.0`, a state-of-the-art embedding model optimized for enterprise search.

Once the search service is defined, you can start querying it with natural language — directly inside Snowflake — without needing external pipelines, vector DBs, or hosting infrastructure.

In [ ]:
-- Create a Cortex Search Service over your chunked transcripts table
CREATE OR REPLACE XXX AICOLLEGE.PUBLIC.TRANSCRIPTS_SEARCH_SERVICE  --> Create "TRANSCRIPTS_SEARCH_SERVICE" via Snowflake's Cortex Search Service hybrid retrieval function
    ON CHUNK
    ATTRIBUTES CUSTOMER_NAME, RELATIVE_PATH
    WAREHOUSE = AICOLLEGE
    TARGET_LAG = '365 days'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'  -- Optional: Select your desired embedding model
    AS (
    SELECT
        CUSTOMER_NAME,
        RELATIVE_PATH,
        CHUNK::VARCHAR AS CHUNK,
    FROM TRANSCRIPT_CHUNKS);

### Test your service — Baseline **“one-shot” RAG** (even with a top-tier model)

Select a customer and then run the *BaselineRAGPipeline* cell to perform the **simplest possible Retrieval-Augmented Generation flow**:

1. **Retrieve** the **first** transcript chunk that semantically matches the user’s question.  
2. **Generate** an answer by stuffing that single chunk into the prompt of a **state-of-the-art model** (*`claude-3-5-sonnet`* or *`mistral-large2`*).

---

> Even with a premium LLM you’ll notice it can only repeat whatever facts happen to live in that lone chunk.  
> It frequently replies with **“Based on the limited context provided …”** because:
>
> * The relevant details might sit in a **different** chunk.  
> * We pass **no metadata** (e.g., speaker tags, meeting header) to help the model reason.  
> * We don’t ask it to **extract** or **structure** anything.
>
> You’ll address all of these gaps in the next steps.

In [ ]:
# Run this and then select a distinct customer from the dropdown list
customers = [r["CUSTOMER_NAME"] for r in session.table("CUSTOMER_INSIGHTS").select("CUSTOMER_NAME").distinct().collect()]

customers = sorted(customers)

selected = st.selectbox(
    "Run this cell, then select a customer to analyze",
    options=customers,
    key="selected_customer"
)

st.success(f"Chosen customer: {st.session_state.selected_customer}")

In [ ]:
# Create service handle
root = Root(get_active_session())
customer = st.session_state.selected_customer
svc  = root.databases["AICOLLEGE"]\
           .schemas["PUBLIC"]\
           .cortex_search_services["XXX"]  # --> Use your Snowflake Cortex Search Service

# Specify user question
question = f"""
            What was the meeting with {customer} about? 
            Who attended? When was the meeting? What is the next step?
            """.strip()

# Get naive retrieval – grab the very first hit (could be just the title page)
hit = svc.search(
        query            = question,
        columns          = ["CUSTOMER_NAME", "CHUNK", "RELATIVE_PATH"],
        limit            = 5,                # look at a few chunks
        search_type      = "embed"           # basic semantic search
     ).results[0]

st.info(f"**File:** {hit['RELATIVE_PATH']}\n\n{hit['CHUNK']}")

# Single-pass generation
response = Complete(
             model  = "XXX",  # --> use Anthropic model or Mistral Large model
             prompt = (
                 f"{question}\n\n"
                 "Answer **only** using the context below:\n"
                 "-----\n"
                 f"{hit['CHUNK']}\n"
                 "-----"
             )
           ).strip()

st.info(f"**LLM Response:**\n\n{response}")

### 🧨 From “one-shot” to 🎯 **Production-ready**: the 3-Step Enhanced RAG Pipeline  

| Step | What happens | Why it matters |
|------|--------------|----------------|
| **1&nbsp;·&nbsp;Smart Retrieval** *(hybrid search + customer filter + top-k)* | We ask `TRANSCRIPTS_SEARCH_SERVICE` for the **five** best chunks that match the user’s question **and** the selected customer.  Hybrid mode blends dense-vector recall *and* keyword precision; score / size filters keep the text relevant and compact. | Pulls **all** salient passages (not just the first hit) while avoiding noise. Higher recall ⇒ fewer “missing-context” answers and less hallucination. |
| **2&nbsp;·&nbsp;Schema-Driven Extraction** *(LLM returns strict JSON)* | A purpose-built prompt instructs the LLM to emit **only** `meeting_date, attendees, agenda, next_step` as JSON.  The prompt includes the concatenated chunks with clear separators. | Guarantees a predictable shape you can trust in code.  Eliminates manual parsing.  Minimises off-topic text & hallucinations. |
| **3&nbsp;·&nbsp;Persist Results** *(Snowpark DataFrame → `TRANSCRIPT_FACTS`)* | The JSON is converted to a Snowpark DF and saved as a table.  Each row is enriched with `customer_name` & `relative_path` so you know exactly where it came from. | Makes the extracted facts **queryable in SQL** immediately. We will JOIN it with your Phase 1 tables or add it to your semantic model.|

---

### Why this 3-Step Enhanced RAG Pipeline beats the “one-shot” Baseline

* **More context ≠ more noise** – Hybrid retrieval surfaces the right chunks, not just the first one.  
* **LLM as parser**, not essayist – A tight schema prompt forces the model to structure data instead of rambling.  
* **SQL-ready output** – Persisting to a temp table lets analysts (or Cortex Analyst) treat unstructured insights just like any other table—no copy-paste required.

> **Bottom line:** this 3-step pipeline turns messy PDF transcripts into reliable, query-ready facts for efficient and accurate results.

In [ ]:
# Build better retrieval (chunk filter + hybrid search) to retrieve top-k chunks for Acme Corp
hits = svc.search(
    query           = question,
    columns         = ["CUSTOMER_NAME", "CHUNK", "RELATIVE_PATH"],
    limit           = 4,                 # grab more context
    search_type     = "hybrid",           # embedding + keyword
    filters         = {"CUSTOMER_NAME": [customer]},  # force-match
    min_score       = 0.15,              # drop low-relevance chunks
    max_chunk_chars = 400                # trim huge blobs
).results

# join them into one context string
context = "\n\n---\n\n".join(f"[{i+1}] {h['CHUNK']}" for i,h in enumerate(hits))
st.info(context)

In [ ]:
# Build structured extraction prompt to return strict JSON so it's easier to load into a table
schema_prompt = f"""
You are a Snowflake analyst. From the context below extract ONLY these fields:
meeting_date, attendees, agenda, next_step.
Return a JSON object with exactly those keys. If a field is missing, use null.

Context:
<<<
{context}
>>>
""".strip()

# call the LLM (no response_format kwarg on this API)
reply_text = XXX(              # --> Use the Snowflake Complete function
    model  = "claude-3-5-sonnet",   # use Anthropic model or Mistral Large model
    prompt = schema_prompt
).strip()

# parse the JSON
facts = json.loads(reply_text)

# enrich
facts["customer_name"]  = customer
facts["relative_path"]  = hits[0]["RELATIVE_PATH"]

# show it
st.json(facts)

In [ ]:
# Persist extracted facts into a table with explicit schema
# Normalise every field that must be STRING
def to_text(v):
    """Convert list/tuple/dict/None to a readable string for Snowflake."""
    if v is None:
        return ""
    if isinstance(v, (list, tuple)):
        # Flatten nested lists first
        flat = []
        for x in v:
            if isinstance(x, (list, tuple)):
                flat.extend(x)
            else:
                flat.append(x)
        return "; ".join(str(x) for x in flat)
    if isinstance(v, dict):
        return json.dumps(v, ensure_ascii=False)
    # already scalar (str, int, float, etc.)
    return str(v)

facts["agenda"]     = to_text(facts.get("agenda"))
facts["next_step"]  = to_text(facts.get("next_step"))

# Ensure attendees is a clean array of strings
att = facts.get("attendees")
if isinstance(att, dict):
    facts["attendees"] = [v for _, v in sorted(att.items(), key=lambda kv: int(kv[0]))]
elif isinstance(att, (list, tuple)):
    facts["attendees"] = [str(a) for a in att]          # coerce to str
else:
    facts["attendees"] = [str(att)] if att is not None else []

# Define the Snowpark schema
fact_schema = T.StructType([
    T.StructField("meeting_date",    T.StringType()),
    T.StructField("attendees",       T.ArrayType(T.StringType())),
    T.StructField("agenda",          T.StringType()),
    T.StructField("next_step",       T.StringType()),
    T.StructField("customer_name",   T.StringType()),
    T.StructField("relative_path",   T.StringType())
])

# Create the DataFrame
snow_df = session.create_dataframe([facts], schema=fact_schema)

# Display in Streamlit
st.markdown("**Structured meeting facts:**")
st.dataframe(snow_df.to_pandas())

# Save as table
snow_df.write.save_as_table("XXX", mode="overwrite", table_type="XXX")  # --> Create a TEMPORARY table called "TRANSCRIPT_FACTS"

st.success("✅ Facts saved to table `TRANSCRIPT_FACTS`")

### Enrich Your Transcript Facts with Sentiment, Key Phrases & Risk Scoring

Before you run the batch pipeline, let’s add three powerful new columns to your `TRANSCRIPT_FACTS` table:

1. **`TRANSCRIPT_SENTIMENT`** – A 0.0–1.0 score capturing the overall tone of the **customer’s dialogue** (timestamps only).  
2. **`KEY_PHRASES`** – An array of distilled phrases representing the customer’s top **use cases**, **pain points**, or **requirements**.  
3. **`RISK_SCORING`** – A 0.0–1.0 numeric indicator of **deal risk**, as assessed by the LLM based on customer language.

These enrichments let you:

- **Track sentiment** by customer or region to spot relationship health.  
- **Filter or group** by specific key phrases (e.g. “Snowpark POC”, “data governance”).  
- **Prioritize follow-ups** by sorting deals by risk score. 

In [ ]:
ALTER TABLE TRANSCRIPT_FACTS 
ADD (
    TRANSCRIPT_SENTIMENT DOUBLE,
    KEY_PHRASES ARRAY,
    RISK_SCORING FLOAT
);

In [ ]:
# Unpack identifiers (from your JSON‐extraction cell) 
customer      = facts["customer_name"]
relative_path = facts["relative_path"]
meeting_date  = facts.get("meeting_date")   # might be None

# Regex to find a “hh:mm” timestamp in the chunk
ts_re = re.compile(r"\b\d{1,2}:\d{2}\b")

dialogue_chunks = []
for h in hits:  
    text = h["CHUNK"]
    m = ts_re.search(text)
    if m:
        # grab from the first timestamp through the end
        dialogue_chunks.append(text[m.start():])
    # else: skip this chunk entirely (it was probably header/overview)

# Fallback if *none* of your hits had timestamps
if dialogue_chunks:
    transcript_text = "\n\n".join(dialogue_chunks)
else:
    # last resort: join all hits (you will see header, but at least you get something)
    transcript_text = "\n\n".join(h["CHUNK"] for h in hits)

st.info(f"✂️ Snippet of transcript used for sentiment (only first 500 chars shown):\n\n{transcript_text[:500]}…")

# Score via a single CORTEX.SENTIMENT() call
sent_score = session.sql(
    "SELECT SNOWFLAKE.XXX(?) AS S",   # --> Use the Snowflake Cortex Sentiment function
    [transcript_text]
).collect()[0]["S"]

# Write back into TRANSCRIPT_FACTS.transcript_sentiment
if meeting_date is not None:
    upd = """
      UPDATE AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS
         SET transcript_sentiment = ?
       WHERE relative_path = ?
         AND customer_name = ?
         AND meeting_date  = ?
    """
    params = [float(sent_score), relative_path, customer, meeting_date]
else:
    upd = """
      UPDATE AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS
         SET transcript_sentiment = ?
       WHERE relative_path = ?
         AND customer_name = ?
    """
    params = [float(sent_score), relative_path, customer]

session.sql(upd, params).collect()
st.success(f"✅ transcript_sentiment = {sent_score:.3f} saved to TRANSCRIPT_FACTS.")

# Verify your update
verify = """
  SELECT meeting_date, customer_name, transcript_sentiment
    FROM AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS
   WHERE relative_path = ?
"""
rows = session.sql(verify, [relative_path]).collect()
st.write(rows)

In [ ]:
# Join your already-sliced transcript dialogue chunks
dialogue_text = "\n\n".join(dialogue_chunks)

# Build a strict extraction prompt
kp_prompt = f"""
You are a Snowflake analyst. From the dialogue below, extract ONLY the key phrases 
that capture what the customer is asking for or highlighting.  

Return a JSON array of strings, nothing else.

Dialogue:
<<<
{dialogue_text}
>>>
""".strip()

# Call Cortex Complete to get back JSON
reply = XXX(                   # --> Use the Snowflake Complete function
    model  = "XXX",   # use Anthropic model or Mistral Large model
    prompt = kp_prompt
).strip()

# Parse JSON array, with a quick fallback to comma-split
try:
    key_phrases = json.loads(reply)
    if not isinstance(key_phrases, list):
        raise ValueError("Expected a JSON list")
    key_phrases = [str(p).strip() for p in key_phrases if p]
except Exception:
    # fallback: strip out brackets and split on commas
    key_phrases = [
        p.strip().strip('"').strip("'")
        for p in reply.strip("[]").split(",")
        if p.strip()
    ]

# ensure we have *some* list
if not isinstance(key_phrases, list):
    key_phrases = []

st.success(f"🔑 Extracted {len(key_phrases)} key-phrases.")

# Persist into TRANSCRIPT_FACTS.key_phrases
up = """
  UPDATE AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS
     SET key_phrases = ?
   WHERE relative_path = ?
     AND customer_name = ?
"""
params = [
    key_phrases,
    relative_path,
    customer
]

session.sql(up, params).collect()
st.success("✅ key_phrases saved to TRANSCRIPT_FACTS.")

# Verify & display the row
verify = """
  SELECT meeting_date, customer_name, key_phrases
    FROM AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS
   WHERE relative_path = ?
"""
row = session.sql(verify, [relative_path]).collect()
st.write(row)

In [ ]:
# Join dialogue as before
dialogue_text = "\n\n".join(dialogue_chunks)

# Build prompt
risk_prompt = f"""
You are a Snowflake analyst assessing customer opportunity risk.  
Based on the dialogue below, return a single numeric risk score between 0.0 (no risk) 
and 1.0 (very high risk), and nothing else.

Dialogue:
<<<
{dialogue_text}
>>>
""".strip()

# Call LLM
risk_reply = XXX(              # --> Use the Snowflake Complete function
    model  = "XXX",   # use Anthropic model or Mistral Large model
    prompt = risk_prompt
).strip()

# Extract the first numeric token
num_match = re.search(r"\d+(?:\.\d+)?", risk_reply)
if not num_match:
    raise ValueError(f"Could not find a numeric risk score in: {risk_reply!r}")
risk_score = float(num_match.group(0))

st.success(f"⚠️ Computed risk score: **{risk_score:.3f}**")

# Persist
upd_sql = """
  UPDATE AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS
     SET risk_scoring = ?
   WHERE relative_path = ?
     AND customer_name = ?
"""
session.sql(upd_sql, [risk_score, relative_path, customer]).collect()
st.success("✅ risk_scoring saved to TRANSCRIPT_FACTS.")

# Verify
verify_sql = """
  SELECT meeting_date, customer_name, risk_scoring
    FROM AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS
   WHERE relative_path = ?
"""
row = session.sql(verify_sql, [relative_path]).collect()
st.write(row)

In [ ]:
SELECT * FROM AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS;

### 🚀 Batch-Process All Discovery Transcripts

In the next step, we’ll run a **single-cell pipeline** that iterates over all 6 customer transcripts and:

1. **Retrieves** the top-*\*k\** chunks for each customer via your `TRANSCRIPTS_SEARCH_SERVICE`.  
2. **Extracts** core meeting facts (`meeting_date`, `attendees`, `agenda`, `next_step`) using a JSON-strict LLM prompt.  
3. **Computes** a **transcript_sentiment** score with `CORTEX.SENTIMENT()`.  
4. **Derives** an array of **key_phrases** via the LLM to capture customer use-cases and pain points.  
5. **Assesses** a numeric **risk_scoring** via the LLM, ranking opportunity health from 0.0 to 1.0.  
6. **Persists** all fields back into your permanent `AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS` table in one go.

By combining retrieval, structured extraction, and LLM-driven enrichment in one cell, you’ll instantly convert raw PDF transcripts into a **searchable, analytics-ready** dataset—no manual joins, no copy-pasting, no intermediate steps.

In [ ]:
# ── Setup
root = Root(session)
svc  = root.databases["AICOLLEGE"] \
           .schemas["PUBLIC"] \
           .cortex_search_services["XXX"]        # --> Use your Snowflake Cortex Search Service

# pull all customer names
customers = [
    r["CUSTOMER_NAME"]
    for r in session
            .table("CUSTOMER_INSIGHTS")
            .select("CUSTOMER_NAME")
            .distinct()
            .collect()
]

# regexes for timestamp & floats
ts_re  = re.compile(r"\b\d{1,2}:\d{2}\b")
num_re = re.compile(r"\d+(?:\.\d+)?")

results = []

# Loop over every customer
for customer in customers:
    question = (
        f"What was the meeting with {customer} about? "
        "Who attended? When was the meeting? What is the next step?"
    )

    # retrieve top‐k context chunks for this customer
    hits = svc.search(
        query           = question,
        columns         = ["CUSTOMER_NAME", "CHUNK", "RELATIVE_PATH"],
        limit           = 10,
        search_type     = "hybrid",
        filters         = {"CUSTOMER_NAME":[customer]},
        min_score       = 0.15,
        max_chunk_chars = 400
    ).results

    if not hits:
        continue

    # Extract core facts via JSON prompt
    context = "\n\n---\n\n".join(f"[{i+1}] {h['CHUNK']}" for i,h in enumerate(hits))
    schema_prompt = f"""
You are a Snowflake analyst. From the context below extract ONLY these fields:
meeting_date, attendees, agenda, next_step.
Return a JSON object with exactly those keys. If a field is missing, use null.

Context:
<<<
{context}
>>>
""".strip()

    reply = Complete(model="mistral-large", prompt=schema_prompt).strip()
    try:
        facts = json.loads(reply)
    except Exception as e:
        print(f"❌ JSON parse failed for {customer}: {e!r}")
        continue

    # add our identifiers
    facts["customer_name"] = customer
    facts["relative_path"] = hits[0]["RELATIVE_PATH"]

    # Build transcript-only text
    dialogue_chunks = []
    for h in hits:
        m = ts_re.search(h["CHUNK"])
        if m:
            dialogue_chunks.append(h["CHUNK"][m.start():])

    transcript_text = (
        "\n\n".join(dialogue_chunks)
        if dialogue_chunks
        else "\n\n".join(h["CHUNK"] for h in hits)
    )

    # Compute transcript_sentiment
    sent = session.sql(
        "SELECT SNOWFLAKE.XXX(?) AS S", [transcript_text]      # --> Use the Snowflake Cortex Sentiment function
    ).collect()[0]["S"]
    facts["transcript_sentiment"] = float(sent)

    # Extract key_phrases via LLM
    kp_prompt = f"""
Extract the key phrases from the following customer dialogue.
Return a JSON array of phrases only, nothing else.

Dialogue:
<<<
{transcript_text}
>>>
""".strip()

    kp_reply = XXX(model="XXX", prompt=kp_prompt).strip()    # --> Use the Snowflake Cortex Complete function
    try:
        kp = json.loads(kp_reply)
        if not isinstance(kp, list):
            raise ValueError("not a list")
        key_phrases = [str(p).strip() for p in kp if p]
    except:
        # fallback to comma‐split
        key_phrases = [
            p.strip().strip('"')
            for p in kp_reply.strip("[]").split(",")
            if p.strip()
        ]
    facts["key_phrases"] = key_phrases

    # Compute risk_scoring via LLM
    risk_prompt = f"""
Based on the following customer dialogue, return a single numeric risk score
between 0.0 (no risk) and 1.0 (very high risk), and nothing else.

Dialogue:
<<<
{transcript_text}
>>>
""".strip()

    risk_reply = XXX(model="XXX", prompt=risk_prompt).strip()    # --> Use the Snowflake Cortex Complete function
    m2 = num_re.search(risk_reply)
    facts["risk_scoring"] = float(m2.group(0)) if m2 else None

    # Collect and move on
    results.append(facts)

# Write ALL results back in one shot
if results:
    df = session.create_dataframe(results)
    df.write.save_as_table("XXX", mode="overwrite")        # --> Create a "TRANSCRIPT_FACTS" table for use with the semantic model
    st.success("✅ All transcripts (+ sentiment, key phrases, risk) saved to TRANSCRIPT_FACTS.")
else:
    st.warning("⚠️ No structured facts extracted; nothing written.")

## 🛠️ Next Steps: Wire `TRANSCRIPT_FACTS` into Your Semantic Model & Search

Now that you’ve batch-loaded all **TRANSCRIPT_FACTS** (core meeting facts + sentiment, key-phrases, risk) into Snowflake, let’s extend your semantic model:

1. **Add `TRANSCRIPT_FACTS` as a Logical Table**
   - In your **data_agent_semantic_model.yaml**, declare a new table mapping to `AICOLLEGE.PUBLIC.TRANSCRIPT_FACTS`.  
   - Define its dimensions (`customer_name`, `meeting_date`, …) and facts (`transcript_sentiment`, `risk_scoring`).  
   - Set the primary key on (`relative_path`, `customer_name`, `meeting_date`).

2. **Create Relationships**  
   - **One-to-many**: link `CUSTOMER_INSIGHTS.meeting_id` → `TRANSCRIPT_FACTS.relative_path` (or `customer_name` + `meeting_date`).  
   - **One-to-one** (optional): connect `CUSTOMER_MEETING_OUTCOMES` ↔ `TRANSCRIPT_FACTS` on (`meeting_id` → `relative_path`).  

3. **Add Verified Queries**  
   - Example: “Which customers have **high transcript sentiment** (>0.5) but **low risk_scoring** (<0.3)?”  
   - Example: “Show me **key phrases** by customer and **agenda** topics.”  
   - Include these under `verified_queries` in your YAML so users can onboard quickly.

4. **Enable Dynamic Literal Retrieval**  
   - For any column with many possible values (e.g. `customer_name`, `snowflake_feature`, `industry`), configure a **Cortex Search Service** in Snowsight:  
     ```sql
     CREATE OR REPLACE CORTEX SEARCH SERVICE AICOLLEGE.PUBLIC.COLUMN_LOOKUP
       ON <COLUMN_VALUE> 
       ATTRIBUTES <COLUMN_NAME>
       WAREHOUSE = AICOLLEGE
       EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
       AS (SELECT DISTINCT <COLUMN_NAME> FROM <TABLE>);
     ```  
   - In the Snowsight UI, link that service under **Dynamic Literal Retrieval** so Analyst will look up filter values at runtime.

5. **Validate & Deploy in Snowsight (No-Code)**  
   - Use the Snowsight **Semantic Model Editor** to import your updated YAML.  
   - Define the new table, relationships, and literal-lookup services.  
   - Publish your changes.

---

🎉  Once the semantic model is updated and deployed, return to **Slack** and try queries like:  
> • “Which customers requested a POC and have transcript_sentiment > 0.5?”  
> • “List risk_scoring by region for all Gen AI discovery meetings.”  

Now you’ll seamlessly span **structured** + **unstructured** insights in one natural-language interface!